In [ ]:
##====================================
##Split Raster
##Usage: SplitRaster_management in_raster out_folder out_base_name SIZE_OF_TILE
##                              | NUMBER_OF_TILES | TIFF | BMP | ENVI | ESRI BIL |
##                              ESRI BIP | ESRI BSQ | GIF | GRID | IMAGINE IMAGE | 
##                              JP2 | JPG | PNG {NEAREST | BILINEAR | CUBIC | 
##                              MAJORITY} {num_rasters} {tile_size} {overlap} 
##                              {PIXELS | METERS | FEET | DEGREES | KILOMETERS | 
##                              MILES} {cell_size} {origin}
    
import arcpy

# Set workspace
arcpy.env.workspace = r"F:/Projekty/DPZ/DPZ.gdb"

##Equally split a large TIFF image by number of images
arcpy.SplitRaster_management("large.tif", "splitras", "number", "NUMBER_OF_TILES",\
                             "TIFF", "NEAREST", "2 2", "#", "4", "PIXELS",\
                             "#", "#")

##Equally split a large TIFF image by size of images
arcpy.SplitRaster_management("large.tif", "splitras", "size2", "SIZE_OF_TILE",\
                             "TIFF", "BILINEAR", "#", "3500 3500", "4", "PIXELS",\
                             "#", "-50 60")

In [4]:
# import libraries
import os
import sys
import arcpy
import numpy as np
import pandas as pd
from arcpy.sa import *
import csv
from PIL import Image

# def createTrainingMosaics(loc, storage_folder, cell_meters, cellSize, composite_array, shift, modification):

gdbPath = r"S:\Private\_PROJEKTY\2020_TACR_DPZ\mapa_udalosti\ryhy_topgis.gdb"
storage_folder = r"S:\Private\_PROJEKTY\2020_TACR_DPZ\mapa_udalosti\Tejkl_reseni"
window = 50
modification = 'none'
composite = "Composites_1\\Topgis10_20190904_Compo1"
composite_ras = arcpy.Raster(composite)
shift = "0"

# analyse it
## cellsize specification
cellSize = composite_ras.meanCellWidth
Cell_Size = str(cellSize)+" "+str(cellSize)

## additional values specification
lowerLeft = arcpy.Point(composite_ras.extent.XMin,composite_ras.extent.YMin)

## run create mosaic not shifted
## export to numpy array
composite_array = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)

# name mosaic, depending on calibration layer
loc = "Topgis10_20190904"

## array dimensions
dimensions = composite_array.shape

# create cell mosaic
cell_meters = window   # width of cell in meters

## copy cell into new raster
pixel_number = int(cell_meters/cellSize)   # pixel number equals to width of cell
pixel_area = pixel_number * pixel_number   # area of cell in pixels

arr_max = []
## array maximums
for band in composite_array:
    arr_max.append(numpy.amax(band))
    
del(arr_max[0])

## print informations
print(cellSize, lowerLeft, dimensions, pixel_number, pixel_area, )

## create greyscale array and append it to composite array
# greyscale_array = (0.3 * Red) + (0.59 * Green) + (0.11 * Blue)
greyscale_array = ((0.3 * composite_array[3]) + (0.59 * composite_array[2]) + (0.11 * composite_array[1]))
composite_array = np.append(composite_array, [greyscale_array], axis=0)

## create log file
log = [[loc, cellSize, lowerLeft, pixel_number, dimensions, shift]]
print(log[0])
## calculate for loop
y = dimensions[1]
x = dimensions[2]

x_steps = int(x/pixel_number)
y_steps = int(y/pixel_number)

if x_steps*cell_meters > x:
    x_steps = x_steps - 1

if y_steps*cell_meters > y:
    y_steps = y_steps - 1

if shift == 1:
    x_steps = x_steps - 1
    y_steps = y_steps - 1

for_range = x_steps * y_steps
print('For range ', for_range)
A = 0
#     for_range = 10000

# mosaic rasters next to each other
mosaic_array = np.full((pixel_number, pixel_number*(dimensions[0]-1)), 0.0)
print('Mosaic shape ',mosaic_array.shape)
counter = 0

k = 210
l = 15

index = 1
s = 0
t = 0
i = 0
x = 0
y = 0

if shift == 1:
    k = int(pixel_number/2)
    l = int(pixel_number/2)
#         print(shift, k, l)

# for step in range(for_range):
n = 1
for band in range(1, dimensions[0]):
    clip_array = composite_array[n][k:k+pixel_number, l:l+pixel_number]
#         print(x, y, l, k, s, s + pixel_number, t, t + pixel_number)

#     print(clip_array)

    with np.nditer(clip_array, op_flags=['readwrite']) as it:
        for pix in it:
            pix[...] = pix * (256/arr_max[n-1])

#     if modification == '90':
#         clip_array = np.rot90(clip_array, k=1)
#     elif modification == '180':
#         clip_array = np.rot90(clip_array, k=2)
#     elif modification == '270':
#         clip_array = np.rot90(clip_array, k=3)
#     elif modification == 'horizontal':
#         clip_array = np.flipud(clip_array)
#     elif modification == 'vertical':
#         clip_array = np.fliplr(clip_array)
#     elif modification == 'none':
#         pass
#     else:
#         pass

    mosaic_array[t:t + pixel_number, s:s + pixel_number] = clip_array
    n += 1
    s = s + pixel_number
    counter += 1
    print(s)

# print(mosaic_array)
print(arr_max)


3.0 -764504,599358347 -1086343,84479847 NaN NaN (6, 1273, 1836) 16 256
['Topgis10_20190904', 3.0, <Point (-764504.5993583474, -1086343.8447984704, #, #)>, 16, (6, 1273, 1836), '0']
For range  8814
Mosaic shape  (16, 80)
16
32
48
64
80
[1263.0, 1383.0, 1841.0, 4998.0, 255.0]


In [6]:
mosaic_array[3]


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [16]:
import numpy as np
import os
from PIL import Image

# composite = "Composites\\Trebesice_20190320_Compo"
# composite_ras = arcpy.Raster(composite)
# composite_array = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)

# arr_max = [numpy.amax(composite_array[1]), numpy.amax(composite_array[2]), numpy.amax(composite_array[3]), numpy.amax(composite_array[4])]

# n = 1
# clip_array = composite_array[n][352:352+16, 528:528+16]
# print(clip_array)

# # with np.nditer(clip_array, op_flags=['readwrite']) as it:
# #     for pix in it:
# #         pix[...] = pix * (256/arr_max[n-1])

# print(clip_array)
# print(arr_max)
# # print(mosaic_array[3])

storage_folder = r"S:\Private\_PROJEKTY\2020_TACR_DPZ\mapa_udalosti\Tejkl_reseni\mosaics"
appendix = '.jpg'
mosaic_save_Rill = os.path.join(storage_folder, 'Pokus' + appendix)
im = Image.fromarray(clip_array)
im = im.convert('RGB')
im.save(mosaic_save_Rill)


In [4]:
# import libraries
import os
import sys
import arcpy
import numpy as np
import pandas as pd
from arcpy.sa import *
import csv
from PIL import Image

# import special libraries
from skimage import feature

def createTrainingMosaics(loc, storage_folder, pixel_number, cellSize, composite_array, shift, modification, just_others):
    
    # verze "just_others" [ryhy, blue, green, red, alpha]
    # verze "just_others" [blue, green, red, alpha]
    A, B = 0, 0
    ## raw composite array dimensions
    dimensions = composite_array.shape
    ## delete pseudo alpha channel
#     composite_array = np.delete(composite_array, dimensions[0]-1, 0)

    ## copy cell into new raster
#     pixel_number = int(cell_meters/cellSize)   # pixel number equals to width of cell
    cell_area = pixel_number * pixel_number   # number of pixels in cell
        
    ## create greyscale array and append it to composite array
    if just_others == "just_others":
        red, green, blue = 0, 1, 2
    else:
        red, green, blue = 1, 2, 3

    # greyscale_array = (0.3 * Red) + (0.59 * Green) + (0.11 * Blue)
    greyscale_array = ((0.3 * composite_array[red]) + (0.59 * composite_array[green]) + (0.11 * composite_array[blue]))
    composite_array = np.append(composite_array, [greyscale_array], axis=0)
    
    # Compute the Canny filter for 3 values of sigma
#     edges1 = feature.canny(greyscale_array)
#     edges2 = feature.canny(greyscale_array, sigma=2)
#     edges3 = feature.canny(greyscale_array, sigma=3)
#     composite_array = np.append(composite_array, [edges1], axis=0)
#     composite_array = np.append(composite_array, [edges2], axis=0)
#     composite_array = np.append(composite_array, [edges3], axis=0)
    
    ## array dimensions
    dimensions = composite_array.shape
    
    ## print informations
    print(cellSize, lowerLeft, dimensions, pixel_number, cell_area, )
    
    ## array maximums
    i = 0
    arr_max = []
    if just_others == "just_others":
        for item in composite_array:
            arr_max.append(numpy.amax(composite_array[i]))
            i += 1
    else:
        for item in composite_array:
            if i == 0:
                i += 1
            else:
                arr_max.append(numpy.amax(composite_array[i]))
                i += 1
                
    print(arr_max)
    
    ## create log file
    log = [[loc, cellSize, lowerLeft, pixel_number, dimensions, shift]]
    print(log[0])
    ## calculate for loop
    y = dimensions[1]
    x = dimensions[2]

    x_steps = int(x/pixel_number)
    y_steps = int(y/pixel_number)

    if x_steps*pixel_number > x:
        x_steps = x_steps - 1

    if y_steps*pixel_number > y:
        y_steps = y_steps - 1
        
    if shift == 1:
        x_steps = x_steps - 1
        y_steps = y_steps - 1

    for_range = x_steps * y_steps
    print('For range ', for_range)
    A = 0
#     for_range = 80

    width = 2
    w = 0
    height = 2
    h = 0
    
    ## create log file
    log[0].append(width)
    log[0].append(height)
    
    ## save metadata if modification is none
    if modification == 'none':
        meta = 'Location: ' + loc + ', Cell Size: ' + str(cellSize) + ', Lower Left: ' + str(lowerLeft) + ', Pixel Number: ' + str(pixel_number) + ', Dimension: ' + str(dimensions) + ', Shift: ' + str(shift) + ', Width: ' + str(width) + ', Height: ' + str(height) + ', '  
        mtd_file = open(os.path.join(storage_folder, 'mosaics', 'MTD_' + loc + '_' + str(shift) +'.txt'),'w')
        mtd_file.write(meta)
        mtd_file.close()

    # mosaic rasters next to each other
#     mosaic_array = np.full((pixel_number, pixel_number*(dimensions[0]-1)), 0.0)
    mosaic_array = np.full((pixel_number * (height), pixel_number * (width)), 0.0)
    print('Empty mosaic array shape ',mosaic_array.shape)
#     print(mosaic_array)
    
    counter = 0
    k = 0
    l = 0
    index = 1
    s = 0
    t = 0
    i = 0
    x = 0
    y = 0
    
    rill_count = 0
    sheet_count = 0
    NoRill_count = 0
    others_count = 0
    
    rill_alternative_list = [0.0, 8, 9]
    
    if shift == 1:
        k = int(pixel_number/2)
        l = int(pixel_number/2)
#         print(shift, k, l)

    for step in range(for_range):
        if just_others == "just_others":
            start_band = 0
            n = 0
            arr_count = 0
        else:
            start_band = 1
            n = 1
            arr_count = 0
            
        for band in range(start_band, dimensions[0]):
            clip_array = composite_array[n][k:k+pixel_number, l:l+pixel_number]
#             print(x, y, l, k, s, s + pixel_number, t, t + pixel_number, w, n, counter)
#             print(clip_array)
    
            with np.nditer(clip_array, op_flags=['readwrite']) as it:
                for pix in it:
                    pix[...] = pix * (256/arr_max[arr_count])
            
            if modification == '90':
                clip_array = np.rot90(clip_array, k=1)
            elif modification == '180':
                clip_array = np.rot90(clip_array, k=2)
            elif modification == '270':
                clip_array = np.rot90(clip_array, k=3)
            elif modification == 'horizontal':
                clip_array = np.flipud(clip_array)
            elif modification == 'vertical':
                clip_array = np.fliplr(clip_array)
            elif modification == 'none':
                pass
            else:
                pass
                    
            mosaic_array[t:t + pixel_number, s:s + pixel_number] = clip_array
            n += 1
            arr_count += 1
            s = s + pixel_number
            w += 1
            if w == width:
#                 print("dosazeno w")
                s = 0
                w = 0
                t = t + pixel_number
#                 print(mosaic_array)
            counter += 1
        
        if just_others == "just_others":
            state_noPix = 0
            edge_array = composite_array[0][k:k+pixel_number, l:l+pixel_number]
            
            # count pixels in masaic based on type
            for item in edge_array:
                for pixel in item:
                    if pixel == 0 or pixel == 255:
                        state_noPix += 1
                    else:
                        pass
                    
            # add suffix based on percentage of pixel types
            if state_noPix > 0.2*cell_area:
                state = "5"
            else:
                state = "0"
            
        else:
            # look for rill signs
            state_noPix = 0
            state_rill = 0
            state_noRill = 0
            state_sheet = 0
            rill_array = composite_array[0][k:k+pixel_number, l:l+pixel_number]
            edge_array = composite_array[1][k:k+pixel_number, l:l+pixel_number]

            # count pixels in masaic based on type
            for item in rill_array:
                for pixel in item:
                    if pixel == 9:
                        state_noRill += 1
                    elif pixel == 8:
                        state_sheet += 1
                    elif pixel not in rill_alternative_list:
                        state_rill += 1
                    else:
                        pass

            # count pixels in masaic based on affinity to edge
            for item in edge_array:
                for pixel in item:
                    if pixel == 0.0:
                        state_noPix += 1
                    elif pixel == 256.0:
                        state_noPix += 1
                    else:
                        pass

            # add suffix based on percentage of pixel types
            if state_noPix > 0.2*cell_area:
                state = "5"
            elif state_rill > 0.001*cell_area:
                state = "1"
            elif state_sheet > 0.1*cell_area:
                state = "8"
            elif state_noRill > 0.1*cell_area:
                state = "9"
            else:
                state = "0"

        X = str(x)
        Y = str(y)
        
        for c in range(for_range):
            if len(X) < len(str(for_range)):
                X = "0" + X
        
        for c in range(for_range):
            if len(Y) < len(str(for_range)):
                Y = "0" + Y
        
        name = loc + "_X" + str(X)  + "Y" + str(Y) + "_" + state + "_" + str(shift)
        
#         if X == "0004" and Y == "0018":
#             print(edge_array[0:10,0:10])
#             print(state_noPix, state_rill, state_sheet, state_noRill, state)
            
        # save mosaic into folder
        # keras takes only jpeg, png, bmp, gif
        appendix = '.jpg'
        mosaic_save_Rill = os.path.join(storage_folder, 'mosaics', 'Rill', "Mosaic_" + name + appendix)
        mosaic_save_Sheet = os.path.join(storage_folder,'mosaics', 'Sheet', "Mosaic_" + name + appendix)
        mosaic_save_NoRill = os.path.join(storage_folder,'mosaics', 'NoRill', "Mosaic_" + name + appendix)
        mosaic_save_others = os.path.join(storage_folder, 'others', "Mosaic_" + name + appendix)

        # save mosaic if it contains data
        if state == "1":
            im = Image.fromarray(mosaic_array)
            im = im.convert('L')
            im.save(mosaic_save_Rill)
            newLine = ["Mosaic_" + name + appendix, X, Y, state, 0, 0 ,0]
            log.append(newLine)
            rill_count += 1
        elif state == "8":
            if modification == 'none':
                im = Image.fromarray(mosaic_array)
                im = im.convert('L')
                im.save(mosaic_save_Sheet)
                newLine = ["Mosaic_" + name + appendix, X, Y, state, 0, 0 ,0]
                log.append(newLine)
                sheet_count += 1 
        elif state == "9":
            if modification == 'none':
                im = Image.fromarray(mosaic_array)
                im = im.convert('L')
                im.save(mosaic_save_NoRill)
                newLine = ["Mosaic_" + name + appendix, X, Y, state, 0, 0 ,0]
                log.append(newLine)
                NoRill_count += 1
        elif state == "0":
            if modification == 'none':
                im = Image.fromarray(mosaic_array)
                im = im.convert('L')
                im.save(mosaic_save_others)
                newLine = ["Mosaic_" + name + appendix, X, Y, state, 0, 0 ,0]
                log.append(newLine)
                others_count += 1
        else:
            newLine = ["Mosaic_" + name + appendix, X, Y, state, 0, 0 ,0]
            pass

        x += 1
        s = 0
        t = 0
        l = l + pixel_number

        # jump to new line
        if x == x_steps:
            x = 0
            l = 0
            k = k + pixel_number
            y += 1
            
            # recognize shift
            if shift == 1:
                l = int(pixel_number/2)
        
#         if state != "5":
#             A += 1
#             
      ##  early end
#         if A == 10:
#             break
            
    ## print out log
    
    ## creation of output file name
    fileName = loc + "_" + str(shift) + "_log.csv"
    outputFileName = os.path.join(storage_folder, 'mosaics', fileName)
    
    if modification == 'none':
        with open(outputFileName, mode='w', encoding = "utf-8") as output_csv:
            csv_writer = csv.writer(output_csv, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            for line in log:
                csv_writer.writerow(line)

    print("Mosaics ", loc, ' shift ', shift, ' done; STATISTICS rills:', rill_count, ', sheets:', sheet_count, ', NoRill', NoRill_count)
    print()

In [5]:
# import libraries
import os
import sys
import arcpy
import numpy as np
import pandas as pd
from arcpy.sa import *

pool = ['33_20201203//Composite_20201203_06', '32_20201126//Composite_20201126_01', '32_20201126//Composite_20201126_02', '32_20201126//Composite_20201126_03', '32_20201126//Composite_20201126_04', 
        '31_20201119//Composite_20201119_01', '31_20201119//Composite_20201119_02', '31_20201119//Composite_20201119_03', '31_20201119//Composite_20201119_04', '31_20201119//Composite_20201119_05', 
        '30_20201112//Composite_20201112_01', '30_20201112//Composite_20201112_02', '30_20201112//Composite_20201112_03', '30_20201112//Composite_20201112_04', '30_20201112//Composite_20201112_05', 
        '29_20201105//Composite_20201105_01', '29_20201105//Composite_20201105_02', '29_20201105//Composite_20201105_03', '28_20201107//Composite_20201102_01', '28_20201106//Composite_20201102_02', 
        '28_20201105//Composite_20201102_06', '28_20201104//Composite_20201102_05', '28_20201103//Composite_20201102_04', '28_20201102//Composite_20201102_03', '27_20200827//Composite_20200827_02', 
        '27_20200827//Composite_20200827_03', '26_20200824//Composite_20200824_01', '26_20200824//Composite_20200824_02', '24_20200810//Composite_20200810_03', '24_20200810//Composite_20200810_01', 
        '23_20200803//Composite_20200803_01', '23_20200803//Composite_20200803_03', '22_20200729//Composite_20200729_01', '22_20200729//Composite_20200729_02', '22_20200729//Composite_20200729_03', 
        '21_20200727//Composite_20200727_01', '21_20200727//Composite_20200727_02', '21_20200727//Composite_20200727_03', '20_20200721//Composite_20200721_01', '20_20200721//Composite_20200721_02', 
        '20_20200721//Composite_20200721_03', '19_20200716//Composite_20200716_01', '19_20200716//Composite_20200716_03', '18_20200708//Composite_20200708_01', '18_20200708//Composite_20200708_02', 
        '17_20190122//Composite_20190122_01', '17_20190122//Composite_20190122_02', '16_20190108//Composite_20190108_01', '16_20190108//Composite_20190108_02', '16_20190108//Composite_20190108_03', 
        '15_20181219//Composite_20181219_01', '15_20181219//Composite_20181219_02', '15_20181219//Composite_20181219_03', '14_20181214//Composite_20181214_01', '14_20181214//Composite_20181214_02', 
        '14_20181214//Composite_20181214_03', '12_20181211//Composite_20181211_04', '12_20181211//Composite_20181211_05', '12_20181211//Composite_20181211_06', '12_20181211//Composite_20181211_01', 
        '12_20181211//Composite_20181211_02', '12_20181211//Composite_20181211_03', '11_20181206//Composite_20181206_01', '11_20181206//Composite_20181206_02', '11_20181206//Composite_20181206_03', 
        '10_20181203//Composite_20181203_01', '10_20181203//Composite_20181203_02', '10_20181203//Composite_20181203_03', '10_20181203//Composite_20181203_04', '09_20181129//Composite_20181129_01', 
        '09_20181129//Composite_20181129_02', '09_20181129//Composite_20181129_03', '08_20181127//Composite_20181127_01', '08_20181127//Composite_20181127_02', '08_20181127//Composite_20181127_03', 
        '05_20181124//Composite_20181124_01', '05_20181124//Composite_20181124_02', '04_20181123//Composite_20181123_02', '04_20181123//Composite_20181123_03', '02_20181115//Composite_20181115_02', 
        '02_20181115//Composite_20181115_03', '01_20181112//Composite_20181112_01', '01_20181112//Composite_20181112_02', '01_20181112//Composite_20181112_03', ]

error_list = []

# take composite
# loc = "Litultovice_20190811"
gdbPath = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\STRIX_ML.gdb"
storage_folder = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\modely\model_011"
window = 50
verze = ""

for item in pool:
    try:
        composite = item
        composite_ras = arcpy.Raster(composite)

        loc = item.split("Composite_")
        loc = loc[1]

        # analyse it
        ## cellsize specification
        cellSize = composite_ras.meanCellWidth
        Cell_Size = str(cellSize)+" "+str(cellSize)

        ## additional values specification
        lowerLeft = arcpy.Point(composite_ras.extent.XMin,composite_ras.extent.YMin)

        ## run create mosaic not shifted
        ## export to numpy array
        composite_array_0 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
        createTrainingMosaics(loc, storage_folder, window, cellSize, composite_array_0, 0, 'none', verze)

        ## run create mosaic shifted
        ## export to numpy array
        composite_array_1 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
        createTrainingMosaics(loc, storage_folder, window, cellSize, composite_array_1, 1, 'none', verze)

        ## run create mosaic turned 90
        ## export to numpy array
        loc = loc + '_A'
        composite_array_2 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
        createTrainingMosaics(loc, storage_folder, window, cellSize, composite_array_2, 0, '90', verze)

        ## run create mosaic turned 180
        ## export to numpy array
        loc = loc.replace('_A', '_B')
        composite_array_3 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
        createTrainingMosaics(loc, storage_folder, window, cellSize, composite_array_3, 0, '180', verze)

        ## run create mosaic turned 180
        ## export to numpy array
        loc = loc.replace('_B', '_C')
        composite_array_4 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
        createTrainingMosaics(loc, storage_folder, window, cellSize, composite_array_4, 0, '270', verze)

        ## run create mosaic flipped horizontally
        ## export to numpy array
        loc = loc.replace('_C', '_D')
        composite_array_5 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
        createTrainingMosaics(loc, storage_folder, window, cellSize, composite_array_5, 0, 'horizontal', verze)

        ## run create mosaic flipped vertically
        ## export to numpy array
        loc = loc.replace('_D', '_E')
        composite_array_6 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
        createTrainingMosaics(loc, storage_folder, window, cellSize, composite_array_6, 0, 'vertical', verze)

        ## run create mosaic turned 90
        ## export to numpy array
        loc = loc.replace('_E', '_F')
        composite_array_7 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
        createTrainingMosaics(loc, storage_folder, window, cellSize, composite_array_7, 1, '90', verze)

        ## run create mosaic turned 180
        ## export to numpy array
        loc = loc.replace('_F', '_G')
        composite_array_8 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
        createTrainingMosaics(loc, storage_folder, window, cellSize, composite_array_8, 1, '180', verze)

        ## run create mosaic turned 180
        ## export to numpy array
        loc = loc.replace('_G', '_H')
        composite_array_9 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
        createTrainingMosaics(loc, storage_folder, window, cellSize, composite_array_9, 1, '270', verze)

        ## run create mosaic flipped horizontally
        ## export to numpy array
        loc = loc.replace('_H', '_I')
        composite_array_10 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
        createTrainingMosaics(loc, storage_folder, window, cellSize, composite_array_10, 1, 'horizontal', verze)

        ## run create mosaic flipped vertically
        ## export to numpy array
        loc = loc.replace('_I', '_J')
        composite_array_11 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
        createTrainingMosaics(loc, storage_folder, window, cellSize, composite_array_11, 1, 'vertical', verze)

    except:
        print("Error ", loc)
        error_list.append(loc)

print(error_list)

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[247.0, 244.0, 242.0, 243.79000000000002]
['20201203_06', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20201203_06  shift  0  done; STATISTICS rills: 0 , sheets: 1483 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[247.0, 244.0, 242.0, 243.79000000000002]
['20201203_06', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 1]
For range  1404
Empty mosaic array shape  (100, 100)
Mosaics  20201203_06  shift  1  done; STATISTICS rills: 0 , sheets: 1385 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[247.0, 244.0, 242.0, 243.79000000000002]
['20201203_06_A', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20201203_06_A  shift  0  done; STATISTICS rills: 0 , sheets: 0 

[247.0, 249.0, 251.0, 247.12]
['20201126_04', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 1]
For range  1404
Empty mosaic array shape  (100, 100)
Mosaics  20201126_04  shift  1  done; STATISTICS rills: 406 , sheets: 445 , NoRill 216

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[247.0, 249.0, 251.0, 247.12]
['20201126_04_A', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20201126_04_A  shift  0  done; STATISTICS rills: 422 , sheets: 0 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[247.0, 249.0, 251.0, 247.12]
['20201126_04_B', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20201126_04_B  shift  0  done; STATISTICS rills: 422 , sheets: 0 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[247.0, 249.0, 2

Mosaics  20201119_04_A  shift  0  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[230.0, 217.0, 215.0, 216.79999999999998]
['20201119_04_B', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20201119_04_B  shift  0  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[230.0, 217.0, 215.0, 216.79999999999998]
['20201119_04_C', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20201119_04_C  shift  0  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[230.0, 217.0, 215.0, 216.79999999999998]
['20201119_04_D', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shap

0.001 0,044 -2,50135450817321E-02 NaN NaN (5, 3978, 984) 50 2500
[255.0, 255.0, 255.0, 255.0]
['20201112_03_C', 0.001, <Point (0.044, -0.025013545081732147, #, #)>, 50, (5, 3978, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20201112_03_C  shift  0  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,50135450817321E-02 NaN NaN (5, 3978, 984) 50 2500
[255.0, 255.0, 255.0, 255.0]
['20201112_03_D', 0.001, <Point (0.044, -0.025013545081732147, #, #)>, 50, (5, 3978, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20201112_03_D  shift  0  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,50135450817321E-02 NaN NaN (5, 3978, 984) 50 2500
[255.0, 255.0, 255.0, 255.0]
['20201112_03_E', 0.001, <Point (0.044, -0.025013545081732147, #, #)>, 50, (5, 3978, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20201112_03_E  shift  0  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,5013545081

0.001 0,044 -2,60000000000002E-02 NaN NaN (5, 3978, 984) 50 2500
[252.0, 251.0, 251.0, 250.75]
['20201105_03_D', 0.001, <Point (0.044, -0.026000000000000245, #, #)>, 50, (5, 3978, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20201105_03_D  shift  0  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,60000000000002E-02 NaN NaN (5, 3978, 984) 50 2500
[252.0, 251.0, 251.0, 250.75]
['20201105_03_E', 0.001, <Point (0.044, -0.026000000000000245, #, #)>, 50, (5, 3978, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20201105_03_E  shift  0  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,60000000000002E-02 NaN NaN (5, 3978, 984) 50 2500
[252.0, 251.0, 251.0, 250.75]
['20201105_03_F', 0.001, <Point (0.044, -0.026000000000000245, #, #)>, 50, (5, 3978, 984), 1]
For range  1404
Empty mosaic array shape  (100, 100)
Mosaics  20201105_03_F  shift  1  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,6000000

...

Mosaics  20181206_03_I  shift  1  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[255.0, 255.0, 255.0, 255.0]
['20181206_03_J', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 1]
For range  1404
Empty mosaic array shape  (100, 100)
Mosaics  20181206_03_J  shift  1  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[254.0, 254.0, 254.0, 251.91999999999996]
['20181203_01', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20181203_01  shift  0  done; STATISTICS rills: 0 , sheets: 0 , NoRill 1258

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[254.0, 254.0, 254.0, 251.91999999999996]
['20181203_01', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 1]
For range  1404
Empty mosaic array shape  (100, 100)
Mo

Mosaics  20181203_04_J  shift  1  done; STATISTICS rills: 62 , sheets: 0 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[255.0, 255.0, 255.0, 255.0]
['20181129_01', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20181129_01  shift  0  done; STATISTICS rills: 0 , sheets: 38 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[255.0, 255.0, 255.0, 255.0]
['20181129_01', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 1]
For range  1404
Empty mosaic array shape  (100, 100)
Mosaics  20181129_01  shift  1  done; STATISTICS rills: 0 , sheets: 39 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[255.0, 255.0, 255.0, 255.0]
['20181129_01_A', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20181129_01_A  shift 

Mosaics  20181127_02  shift  1  done; STATISTICS rills: 0 , sheets: 162 , NoRill 483

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[255.0, 255.0, 255.0, 255.0]
['20181127_02_A', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20181127_02_A  shift  0  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[255.0, 255.0, 255.0, 255.0]
['20181127_02_B', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20181127_02_B  shift  0  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[255.0, 255.0, 255.0, 255.0]
['20181127_02_C', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20181127_02_C 

For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20181123_02_B  shift  0  done; STATISTICS rills: 466 , sheets: 0 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[255.0, 255.0, 255.0, 255.0]
['20181123_02_C', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20181123_02_C  shift  0  done; STATISTICS rills: 466 , sheets: 0 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[255.0, 255.0, 255.0, 255.0]
['20181123_02_D', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20181123_02_D  shift  0  done; STATISTICS rills: 466 , sheets: 0 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[255.0, 255.0, 255.0, 255.0]
['20181123_02_E', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Emp

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[254.0, 251.0, 252.0, 250.81]
['20181112_01_D', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20181112_01_D  shift  0  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[254.0, 251.0, 252.0, 250.81]
['20181112_01_E', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 0]
For range  1501
Empty mosaic array shape  (100, 100)
Mosaics  20181112_01_E  shift  0  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,50000000000004E-02 NaN NaN (5, 3977, 984) 50 2500
[254.0, 251.0, 252.0, 250.81]
['20181112_01_F', 0.001, <Point (0.044, -0.025000000000000355, #, #)>, 50, (5, 3977, 984), 1]
For range  1404
Empty mosaic array shape  (100, 100)
Mosaics  20181112_01_F  shift  1  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0,044 -2,5000000

In [7]:
# import libraries
import os
import sys
import arcpy
import numpy as np
import pandas as pd
from arcpy.sa import *

# pool = [ "ds_20201126_puda_01", "ds_20201126_puda_02", "ds_20201126_puda_03", "ds_20201126_puda_04"]
pool = [ "ORTHO_20220713_01.tif", "ORTHO_20220713_02.tif"]

# take composite
# loc = "Litultovice_20190811"
gdbPath = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\STRIX_ML.gdb"
storage_folder = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\modely\model_011"
window = 50
verze = "just_others"

for loc in pool:
#     composite = "DS\\" + loc + ".tif"
    composite = "Annete\\" + loc
    composite_ras = arcpy.Raster(composite)
    name = loc.split("_")
    loc = "ds_" + name[1] + "_" + name[2]
    loc = loc.replace(".tif","")

    # analyse it
    ## cellsize specification
    cellSize = composite_ras.meanCellWidth
    Cell_Size = str(cellSize)+" "+str(cellSize)

    ## additional values specification
    lowerLeft = arcpy.Point(composite_ras.extent.XMin,composite_ras.extent.YMin)

    ## run create mosaic not shifted
    ## export to numpy array
    composite_array_0 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
    createTrainingMosaics(loc, storage_folder, window, cellSize, composite_array_0, 0, 'none', verze)

    ## run create mosaic shifted
    ## export to numpy array
    composite_array_1 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
    createTrainingMosaics(loc, storage_folder, window, cellSize, composite_array_1, 1, 'none', verze)

0.001 0 -4,92793900000001E-02 NaN NaN (4, 4049, 1070) 50 2500
[185.0, 192.0, 207.0, 189.66]
['ds_20220713_01', 0.001, <Point (0.0, -0.049279390000000145, #, #)>, 50, (4, 4049, 1070), 0]
For range  1680
Empty mosaic array shape  (100, 100)
Mosaics  ds_20220713_01  shift  0  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0 -4,92793900000001E-02 NaN NaN (4, 4049, 1070) 50 2500
[185.0, 192.0, 207.0, 189.66]
['ds_20220713_01', 0.001, <Point (0.0, -0.049279390000000145, #, #)>, 50, (4, 4049, 1070), 1]
For range  1580
Empty mosaic array shape  (100, 100)
Mosaics  ds_20220713_01  shift  1  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0 -4,92397760000003E-02 NaN NaN (4, 4049, 1070) 50 2500
[237.0, 244.0, 243.0, 240.88999999999996]
['ds_20220713_02', 0.001, <Point (0.0, -0.04923977600000029, #, #)>, 50, (4, 4049, 1070), 0]
For range  1680
Empty mosaic array shape  (100, 100)
Mosaics  ds_20220713_02  shift  0  done; STATISTICS rills: 0 , sheets: 0 , NoRill 0

0.001 0 -4,923977

In [58]:
composite = "Composites_R1\\biomac_c_R1_12_2_02_compo"
state_noPix = 0
pixel_number = 50
X, Y = 4, 18
k, l = Y*pixel_number, X*pixel_number
state = "0"
cell_area = pixel_number * pixel_number
composite_ras = arcpy.Raster(composite)
composite_array_0 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
edge_array = composite_array_0[1][k:k+pixel_number, l:l+pixel_number]
for item in edge_array:
    for pixel in item:
        if pixel == 0:
            state_noPix += 1
        elif pixel == 255:
            state_noPix += 1
        else:
            pass

if state_noPix > 0.2*cell_area:
    state = "5"
else:
    state = "1"
print(state_noPix, state, 0.2*cell_area)

2500 5 500.0


In [25]:
composite_ras = arcpy.Raster("24_20200810//Composite_20200810_03")
loc = "20200810_03"
# analyse it
## cellsize specification
cellSize = composite_ras.meanCellWidth
Cell_Size = str(cellSize)+" "+str(cellSize)

## additional values specification
lowerLeft = arcpy.Point(composite_ras.extent.XMin,composite_ras.extent.YMin)

## run create mosaic not shifted
## export to numpy array
composite_array_0 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
createTrainingMosaics(loc, storage_folder, window, cellSize, composite_array_0, 0, 'none', verze)

0.001 4,36792573014939E-02 -2,51965387760675E-02 NaN NaN (5, 3978, 984) 50 2500
[254.0, 253.0, 254.0, 253.10999999999996]
['20200810_03', 0.001, <Point (0.04367925730149386, -0.025196538776067534, #, #)>, 50, (5, 3978, 984), 0]
For range  1501
Empty mosaic array shape  (300, 250)
Mosaics  20200810_03  shift  0  done



In [8]:
# import libraries
import os
import sys
import arcpy
import numpy as np
import pandas as pd
from arcpy.sa import *
import csv

# load log and create raster

def createRillRaster(name, folderPath, output_folder):

    # log name
#     name = "Trebesice_20170815_0_log.csv"

    # folder adress

#     folderPath = r"S:\Private\_PROJEKTY\2020_TACR_DPZ\mapa_udalosti\Tejkl_reseni\mosaics"
    logPath = os.path.join(folderPath, name)

    # load csv
    log = []
    with open(logPath , newline = '',encoding= 'utf-8') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=';', quotechar='"')
        for row in csv_reader:
            log.append(row)

    newList = []
    for line in log:
        if line == []:
            pass
        else:
            newList.append(line)

    log = newList

    # get metadata data
    loc = log[0][0]

    cellSize = float(log[0][1])
    Cell_Size = str(cellSize)+" "+str(cellSize)

    lowerLeft = log[0][2]
    lowerLeft = lowerLeft.split(' ')
    lowerLeft = arcpy.Point(lowerLeft[0], lowerLeft[1])

    pixel_number = int(log[0][3])

    dimensions = log[0][4]
    dimensions = dimensions.replace('(', '')
    dimensions = dimensions.replace(')', '')
    dimensions = dimensions.split(',')

    temp = []
    for item in dimensions:
        temp.append(int(item))
    dimensions = temp
    shift = int(log[0][5])

    # create empty raster
    NoRillArray = numpy.zeros((dimensions[1], dimensions[2]))
    rillArray = numpy.zeros((dimensions[1], dimensions[2]))
    sheetArray = numpy.zeros((dimensions[1], dimensions[2]))
    
    # recognize shift
    s, t = 0, 0
    if shift == 1:
        s = int(pixel_number/2)
        t = int(pixel_number/2)
    
    # iterate over the log and fil the raster
    i = 0
    NoRill = 0.0
    rill = 0.0
    sheet = 0.0
    
    for line in log:
#         print(line)
        if i == 0:
            pass
        else:
            X = int(line[1])
            Y = int(line[2])
            NoRill = float(line[4])
            rill = float(line[5])
            sheet = float(line[6])
#             print(result)

            k = Y * pixel_number + s
            l = X * pixel_number + t
            NoRillArray[k:k+pixel_number, l:l+pixel_number] = NoRill
            rillArray[k:k+pixel_number, l:l+pixel_number] = rill
            sheetArray[k:k+pixel_number, l:l+pixel_number] = sheet

        i += 1

#     return(output_array)
    NoRillRaster = arcpy.NumPyArrayToRaster(NoRillArray,lowerLeft,cellSize, value_to_nodata=255)
    rillRaster = arcpy.NumPyArrayToRaster(rillArray,lowerLeft,cellSize, value_to_nodata=255)
    sheetRaster = arcpy.NumPyArrayToRaster(sheetArray,lowerLeft,cellSize, value_to_nodata=255)
    
    NoRasName = name.replace('_result.csv', '_no.tif')
    rillRasName = name.replace('_result.csv', '_rill.tif')
    sheetRasName = name.replace('_result.csv', '_sheet.tif')
    
    print(rillRasName)

    NoRillRaster.save(os.path.join(output_folder, NoRasName))
    rillRaster.save(os.path.join(output_folder, rillRasName))
    sheetRaster.save(os.path.join(output_folder, sheetRasName))

    print('Rill Raster for ', name, ' done')  
             

In [10]:
# import libraries
import os
import sys
import arcpy
import numpy as np
import pandas as pd
from arcpy.sa import *
import csv

# name = "Bozetin_20200801_0_result.csv"
folderPath = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\modely\model_011\mosaics"
output_folder = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\modely\model_011\result_rasters"

log_list = ['20181112_01_0_log.csv', '20181112_01_1_log.csv', '20181112_02_0_log.csv', '20181112_02_1_log.csv', '20181112_03_0_log.csv', '20181112_03_1_log.csv', '20181115_02_0_log.csv', '20181115_02_1_log.csv', '20181115_03_0_log.csv', '20181115_03_1_log.csv', '20181123_02_0_log.csv', '20181123_02_1_log.csv',
            '20181123_03_0_log.csv', '20181123_03_1_log.csv', '20181124_01_0_log.csv', '20181124_01_1_log.csv', '20181124_02_0_log.csv', '20181124_02_1_log.csv', '20181127_01_0_log.csv', '20181127_01_1_log.csv', '20181127_02_0_log.csv', '20181127_02_1_log.csv', '20181127_03_0_log.csv', '20181127_03_1_log.csv',
            '20181129_01_0_log.csv', '20181129_01_1_log.csv', '20181129_02_0_log.csv', '20181129_02_1_log.csv', '20181129_03_0_log.csv', '20181129_03_1_log.csv', '20181203_01_0_log.csv', '20181203_01_1_log.csv', '20181203_02_0_log.csv', '20181203_02_1_log.csv', '20181203_03_0_log.csv', '20181203_03_1_log.csv',
            '20181203_04_0_log.csv', '20181203_04_1_log.csv', '20181206_01_0_log.csv', '20181206_01_1_log.csv', '20181206_02_0_log.csv', '20181206_02_1_log.csv', '20181206_03_0_log.csv', '20181206_03_1_log.csv', '20181211_01_0_log.csv', '20181211_01_1_log.csv', '20181211_02_0_log.csv', '20181211_02_1_log.csv',
            '20181211_03_0_log.csv', '20181211_03_1_log.csv', '20181211_04_0_log.csv', '20181211_04_1_log.csv', '20181211_05_0_log.csv', '20181211_05_1_log.csv', '20181211_06_0_log.csv', '20181211_06_1_log.csv', '20181214_01_0_log.csv', '20181214_01_1_log.csv', '20181214_02_0_log.csv', '20181214_02_1_log.csv',
            '20181214_03_0_log.csv', '20181214_03_1_log.csv', '20181219_01_0_log.csv', '20181219_01_1_log.csv', '20181219_02_0_log.csv', '20181219_02_1_log.csv', '20181219_03_0_log.csv', '20181219_03_1_log.csv', '20190108_01_0_log.csv', '20190108_01_1_log.csv', '20190108_02_0_log.csv', '20190108_02_1_log.csv',
            '20190108_03_0_log.csv', '20190108_03_1_log.csv', '20200708_01_0_log.csv', '20200708_01_1_log.csv', '20200708_02_0_log.csv', '20200708_02_1_log.csv', '20200716_01_0_log.csv', '20200716_01_1_log.csv', '20200716_03_0_log.csv', '20200716_03_1_log.csv', '20200721_01_0_log.csv', '20200721_01_1_log.csv',
            '20200721_02_0_log.csv', '20200721_02_1_log.csv', '20200721_03_0_log.csv', '20200721_03_1_log.csv', '20200727_01_0_log.csv', '20200727_01_1_log.csv', '20200727_02_0_log.csv', '20200727_02_1_log.csv', '20200727_03_0_log.csv', '20200727_03_1_log.csv', '20200729_01_0_log.csv', '20200729_01_1_log.csv',
            '20200729_02_0_log.csv', '20200729_02_1_log.csv', '20200729_03_0_log.csv', '20200729_03_1_log.csv', '20200803_01_0_log.csv', '20200803_01_1_log.csv', '20200803_03_0_log.csv', '20200803_03_1_log.csv', '20200810_01_0_log.csv', '20200810_01_1_log.csv', '20200810_03_0_log.csv', '20200810_03_1_log.csv',
            '20200824_01_0_log.csv', '20200824_01_1_log.csv', '20200824_02_0_log.csv', '20200824_02_1_log.csv', '20200827_02_0_log.csv', '20200827_02_1_log.csv', '20200827_03_0_log.csv', '20200827_03_1_log.csv', '20201102_03_0_log.csv', '20201102_03_1_log.csv', '20201105_01_0_log.csv', '20201105_01_1_log.csv',
            '20201105_02_0_log.csv', '20201105_02_1_log.csv', '20201105_03_0_log.csv', '20201105_03_1_log.csv', '20201112_01_0_log.csv', '20201112_01_1_log.csv', '20201112_02_0_log.csv', '20201112_02_1_log.csv', '20201112_03_0_log.csv', '20201112_03_1_log.csv', '20201112_04_0_log.csv', '20201112_04_1_log.csv',
            '20201119_01_0_log.csv', '20201119_01_1_log.csv', '20201119_02_0_log.csv', '20201119_02_1_log.csv', '20201119_03_0_log.csv', '20201119_03_1_log.csv', '20201119_04_0_log.csv', '20201119_04_1_log.csv', '20201119_05_0_log.csv', '20201119_05_1_log.csv', '20201126_01_0_log.csv', '20201126_01_1_log.csv',
            '20201126_02_0_log.csv', '20201126_02_1_log.csv', '20201126_03_0_log.csv', '20201126_03_1_log.csv', '20201126_04_0_log.csv', '20201126_04_1_log.csv', '20201203_06_0_log.csv', '20201203_06_1_log.csv']
log_list = ['ds_20220713_01_0_log.csv', 'ds_20220713_01_1_log.csv', 'ds_20220713_02_0_log.csv', 'ds_20220713_02_1_log.csv']

# A = [0, 1]
# for item in A:
#     if item == 1:
#         name = name.replace('_0_', '_1_')

for item in log_list:
    name = item.replace('_log', '_result')
    createRillRaster(name, folderPath, output_folder)
    
#     item_split = item.split("_")
#     name_0 = item_split[0] + "_" + item_split[1] + "_0_" + item_split[3]
#     temp_0 =  createRillRaster(name_0, folderPath)
    
#     logPath = os.path.join(folderPath, name_0)

#     # load csv
#     log = []
#     with open(logPath , newline = '',encoding= 'utf-8') as csvfile:
#         csv_reader = csv.reader(csvfile, delimiter=';', quotechar='"')
#         for row in csv_reader:
#             log.append(row)

#     newList = []
#     for line in log:
#         if line == []:
#             pass
#         else:
#             newList.append(line)

#     log = newList

#     # get metadata data
#     loc = log[0][0]
    
#     cellSize = float(log[0][1])
#     Cell_Size = str(cellSize)+" "+str(cellSize)

#     lowerLeft = log[0][2]
#     lowerLeft = lowerLeft.split(' ')
#     lowerLeft = arcpy.Point(lowerLeft[0], lowerLeft[1])

#     name_1 = item_split[0] + "_" + item_split[1] + "_1_" + item_split[3]
#     temp_1 = createRillRaster(name_1, folderPath)
    
#     resultArray = (temp_0 + temp_1)/2
#     newRaster = arcpy.NumPyArrayToRaster(resultArray,lowerLeft,cellSize, value_to_nodata=255)
#     rasterName = item_split[0] + "_" + item_split[1] + "_avg"
#     newRaster.save(os.path.join(output_folder, rasterName))


ds_20220713_01_0_rill.tif
Rill Raster for  ds_20220713_01_0_result.csv  done
ds_20220713_01_1_rill.tif
Rill Raster for  ds_20220713_01_1_result.csv  done
ds_20220713_02_0_rill.tif
Rill Raster for  ds_20220713_02_0_result.csv  done
ds_20220713_02_1_rill.tif
Rill Raster for  ds_20220713_02_1_result.csv  done


In [11]:
import arcpy
from arcpy.ia import *
from sys import argv
import os

log_list = ['20181112_01_0_log.csv', '20181112_01_1_log.csv', '20181112_02_0_log.csv', '20181112_02_1_log.csv', '20181112_03_0_log.csv', '20181112_03_1_log.csv', '20181115_02_0_log.csv', '20181115_02_1_log.csv', '20181115_03_0_log.csv', '20181115_03_1_log.csv', '20181123_02_0_log.csv', '20181123_02_1_log.csv',
            '20181123_03_0_log.csv', '20181123_03_1_log.csv', '20181124_01_0_log.csv', '20181124_01_1_log.csv', '20181124_02_0_log.csv', '20181124_02_1_log.csv', '20181127_01_0_log.csv', '20181127_01_1_log.csv', '20181127_02_0_log.csv', '20181127_02_1_log.csv', '20181127_03_0_log.csv', '20181127_03_1_log.csv',
            '20181129_01_0_log.csv', '20181129_01_1_log.csv', '20181129_02_0_log.csv', '20181129_02_1_log.csv', '20181129_03_0_log.csv', '20181129_03_1_log.csv', '20181203_01_0_log.csv', '20181203_01_1_log.csv', '20181203_02_0_log.csv', '20181203_02_1_log.csv', '20181203_03_0_log.csv', '20181203_03_1_log.csv',
            '20181203_04_0_log.csv', '20181203_04_1_log.csv', '20181206_01_0_log.csv', '20181206_01_1_log.csv', '20181206_02_0_log.csv', '20181206_02_1_log.csv', '20181206_03_0_log.csv', '20181206_03_1_log.csv', '20181211_01_0_log.csv', '20181211_01_1_log.csv', '20181211_02_0_log.csv', '20181211_02_1_log.csv',
            '20181211_03_0_log.csv', '20181211_03_1_log.csv', '20181211_04_0_log.csv', '20181211_04_1_log.csv', '20181211_05_0_log.csv', '20181211_05_1_log.csv', '20181211_06_0_log.csv', '20181211_06_1_log.csv', '20181214_01_0_log.csv', '20181214_01_1_log.csv', '20181214_02_0_log.csv', '20181214_02_1_log.csv',
            '20181214_03_0_log.csv', '20181214_03_1_log.csv', '20181219_01_0_log.csv', '20181219_01_1_log.csv', '20181219_02_0_log.csv', '20181219_02_1_log.csv', '20181219_03_0_log.csv', '20181219_03_1_log.csv', '20190108_01_0_log.csv', '20190108_01_1_log.csv', '20190108_02_0_log.csv', '20190108_02_1_log.csv',
            '20190108_03_0_log.csv', '20190108_03_1_log.csv', '20200708_01_0_log.csv', '20200708_01_1_log.csv', '20200708_02_0_log.csv', '20200708_02_1_log.csv', '20200716_01_0_log.csv', '20200716_01_1_log.csv', '20200716_03_0_log.csv', '20200716_03_1_log.csv', '20200721_01_0_log.csv', '20200721_01_1_log.csv',
            '20200721_02_0_log.csv', '20200721_02_1_log.csv', '20200721_03_0_log.csv', '20200721_03_1_log.csv', '20200727_01_0_log.csv', '20200727_01_1_log.csv', '20200727_02_0_log.csv', '20200727_02_1_log.csv', '20200727_03_0_log.csv', '20200727_03_1_log.csv', '20200729_01_0_log.csv', '20200729_01_1_log.csv',
            '20200729_02_0_log.csv', '20200729_02_1_log.csv', '20200729_03_0_log.csv', '20200729_03_1_log.csv', '20200803_01_0_log.csv', '20200803_01_1_log.csv', '20200803_03_0_log.csv', '20200803_03_1_log.csv', '20200810_01_0_log.csv', '20200810_01_1_log.csv', '20200810_03_0_log.csv', '20200810_03_1_log.csv',
            '20200824_01_0_log.csv', '20200824_01_1_log.csv', '20200824_02_0_log.csv', '20200824_02_1_log.csv', '20200827_02_0_log.csv', '20200827_02_1_log.csv', '20200827_03_0_log.csv', '20200827_03_1_log.csv', '20201102_03_0_log.csv', '20201102_03_1_log.csv', '20201105_01_0_log.csv', '20201105_01_1_log.csv',
            '20201105_02_0_log.csv', '20201105_02_1_log.csv', '20201105_03_0_log.csv', '20201105_03_1_log.csv', '20201112_01_0_log.csv', '20201112_01_1_log.csv', '20201112_02_0_log.csv', '20201112_02_1_log.csv', '20201112_03_0_log.csv', '20201112_03_1_log.csv', '20201112_04_0_log.csv', '20201112_04_1_log.csv',
            '20201119_01_0_log.csv', '20201119_01_1_log.csv', '20201119_02_0_log.csv', '20201119_02_1_log.csv', '20201119_03_0_log.csv', '20201119_03_1_log.csv', '20201119_04_0_log.csv', '20201119_04_1_log.csv', '20201119_05_0_log.csv', '20201119_05_1_log.csv', '20201126_01_0_log.csv', '20201126_01_1_log.csv',
            '20201126_02_0_log.csv', '20201126_02_1_log.csv', '20201126_03_0_log.csv', '20201126_03_1_log.csv', '20201126_04_0_log.csv', '20201126_04_1_log.csv', '20201203_06_0_log.csv', '20201203_06_1_log.csv']
log_list = ['ds_20220713_01_0_log.csv', 'ds_20220713_01_1_log.csv', 'ds_20220713_02_0_log.csv', 'ds_20220713_02_1_log.csv']

for item in log_list:
    
    item_split = item.split("_")
    if item_split[3] == "0":
        name_0 = item_split[0] + "_" + item_split[1] + "_" + item_split[2] + "_0.tif"
        name_1 = item_split[0] + "_" + item_split[1] + "_" + item_split[2] + "_1.tif"
        result_name = item_split[0] + "_" + item_split[1] + "_" + item_split[2] + "_avg.tif"

        Input_1 = os.path.join("Results_1", name_0)
        Input_2 = os.path.join("Results_1", name_1)
        Result_raster = os.path.join(" S:\\Private\\_PROJEKTY\\2017_Strix_svahy\\Tejkl_ML\\machine_learning\\modely\\model_011\\result_rasters", result_name)
        # To allow overwriting outputs change overwriteOutput option to True.
        arcpy.env.overwriteOutput = True

        # Check out any necessary licenses.
        arcpy.CheckOutExtension("spatial")

        # Process: Raster Calculator (Raster Calculator) (ia)
        projection = "PROJCS['S-JTSK_Krovak_East_North',GEOGCS['GCS_S_JTSK',DATUM['D_S_JTSK',SPHEROID['Bessel_1841',6377397.155,299.1528128]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Krovak'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Pseudo_Standard_Parallel_1',78.5],PARAMETER['Scale_Factor',0.9999],PARAMETER['Azimuth',30.28813975277778],PARAMETER['Longitude_Of_Center',24.83333333333333],PARAMETER['Latitude_Of_Center',49.5],PARAMETER['X_Scale',-1.0],PARAMETER['Y_Scale',1.0],PARAMETER['XY_Plane_Rotation',90.0],UNIT['Meter',1.0]]"

        # Set local variables
        in_raster1 = "raster1.tif"
        in_raster2 = "raster2.tif"

        # Excuate RasterCalculator(Minus) function
        raster_calc = RasterCalculator([Input_1, Input_2], ["x", "y"], "(x+y)/2", "FirstOf", "FirstOf")

        arcpy.management.DefineProjection(raster_calc, projection)

        # Save the output
        raster_calc.save(Result_raster)
        print("Raster ", result_name, " done")
        
print("Finished")




RuntimeError: Failed to apply Raster Function: 'RasterCalculator' (Parametr není správný. 
Parameter 'Rasters' is missing or invalid. Bind failed in function 'Raster Calculator Function' [Raster Calculator Function].)

In [15]:
import arcpy
from arcpy.ia import *
from sys import argv
import os

# average of multiple inputs

log_list = ['20181112_01_0_log.csv', '20181112_01_1_log.csv', '20181112_02_0_log.csv', '20181112_02_1_log.csv', '20181112_03_0_log.csv', '20181112_03_1_log.csv', '20181115_02_0_log.csv', '20181115_02_1_log.csv', '20181115_03_0_log.csv', '20181115_03_1_log.csv', '20181123_02_0_log.csv', '20181123_02_1_log.csv',
            '20181123_03_0_log.csv', '20181123_03_1_log.csv', '20181124_01_0_log.csv', '20181124_01_1_log.csv', '20181124_02_0_log.csv', '20181124_02_1_log.csv', '20181127_01_0_log.csv', '20181127_01_1_log.csv', '20181127_02_0_log.csv', '20181127_02_1_log.csv', '20181127_03_0_log.csv', '20181127_03_1_log.csv',
            '20181129_01_0_log.csv', '20181129_01_1_log.csv', '20181129_02_0_log.csv', '20181129_02_1_log.csv', '20181129_03_0_log.csv', '20181129_03_1_log.csv', '20181203_01_0_log.csv', '20181203_01_1_log.csv', '20181203_02_0_log.csv', '20181203_02_1_log.csv', '20181203_03_0_log.csv', '20181203_03_1_log.csv',
            '20181203_04_0_log.csv', '20181203_04_1_log.csv', '20181206_01_0_log.csv', '20181206_01_1_log.csv', '20181206_02_0_log.csv', '20181206_02_1_log.csv', '20181206_03_0_log.csv', '20181206_03_1_log.csv', '20181211_01_0_log.csv', '20181211_01_1_log.csv', '20181211_02_0_log.csv', '20181211_02_1_log.csv',
            '20181211_03_0_log.csv', '20181211_03_1_log.csv', '20181211_04_0_log.csv', '20181211_04_1_log.csv', '20181211_05_0_log.csv', '20181211_05_1_log.csv', '20181211_06_0_log.csv', '20181211_06_1_log.csv', '20181214_01_0_log.csv', '20181214_01_1_log.csv', '20181214_02_0_log.csv', '20181214_02_1_log.csv',
            '20181214_03_0_log.csv', '20181214_03_1_log.csv', '20181219_01_0_log.csv', '20181219_01_1_log.csv', '20181219_02_0_log.csv', '20181219_02_1_log.csv', '20181219_03_0_log.csv', '20181219_03_1_log.csv', '20190108_01_0_log.csv', '20190108_01_1_log.csv', '20190108_02_0_log.csv', '20190108_02_1_log.csv',
            '20190108_03_0_log.csv', '20190108_03_1_log.csv', '20200708_01_0_log.csv', '20200708_01_1_log.csv', '20200708_02_0_log.csv', '20200708_02_1_log.csv', '20200716_01_0_log.csv', '20200716_01_1_log.csv', '20200716_03_0_log.csv', '20200716_03_1_log.csv', '20200721_01_0_log.csv', '20200721_01_1_log.csv',
            '20200721_02_0_log.csv', '20200721_02_1_log.csv', '20200721_03_0_log.csv', '20200721_03_1_log.csv', '20200727_01_0_log.csv', '20200727_01_1_log.csv', '20200727_02_0_log.csv', '20200727_02_1_log.csv', '20200727_03_0_log.csv', '20200727_03_1_log.csv', '20200729_01_0_log.csv', '20200729_01_1_log.csv',
            '20200729_02_0_log.csv', '20200729_02_1_log.csv', '20200729_03_0_log.csv', '20200729_03_1_log.csv', '20200803_01_0_log.csv', '20200803_01_1_log.csv', '20200803_03_0_log.csv', '20200803_03_1_log.csv', '20200810_01_0_log.csv', '20200810_01_1_log.csv', '20200810_03_0_log.csv', '20200810_03_1_log.csv',
            '20200824_01_0_log.csv', '20200824_01_1_log.csv', '20200824_02_0_log.csv', '20200824_02_1_log.csv', '20200827_02_0_log.csv', '20200827_02_1_log.csv', '20200827_03_0_log.csv', '20200827_03_1_log.csv', '20201102_03_0_log.csv', '20201102_03_1_log.csv', '20201105_01_0_log.csv', '20201105_01_1_log.csv',
            '20201105_02_0_log.csv', '20201105_02_1_log.csv', '20201105_03_0_log.csv', '20201105_03_1_log.csv', '20201112_01_0_log.csv', '20201112_01_1_log.csv', '20201112_02_0_log.csv', '20201112_02_1_log.csv', '20201112_03_0_log.csv', '20201112_03_1_log.csv', '20201112_04_0_log.csv', '20201112_04_1_log.csv',
            '20201119_01_0_log.csv', '20201119_01_1_log.csv', '20201119_02_0_log.csv', '20201119_02_1_log.csv', '20201119_03_0_log.csv', '20201119_03_1_log.csv', '20201119_04_0_log.csv', '20201119_04_1_log.csv', '20201119_05_0_log.csv', '20201119_05_1_log.csv', '20201126_01_0_log.csv', '20201126_01_1_log.csv',
            '20201126_02_0_log.csv', '20201126_02_1_log.csv', '20201126_03_0_log.csv', '20201126_03_1_log.csv', '20201126_04_0_log.csv', '20201126_04_1_log.csv', '20201203_06_0_log.csv', '20201203_06_1_log.csv']
log_list = ['ds_20220713_01_0_log.csv', 'ds_20220713_01_1_log.csv', 'ds_20220713_02_0_log.csv', 'ds_20220713_02_1_log.csv']

# appendix rill, no, sheet
appendix_list = ["rill", "no", "sheet"]
file_path = r"S:\\Private\\_PROJEKTY\\2017_Strix_svahy\\Tejkl_ML\\machine_learning\\modely\\model_011\\result_rasters"
error_list = []

for appendix in appendix_list:
    for item in log_list:

        try:
            item_split = item.split("_")
            if item_split[3] == "0":
#                 name_0 = item_split[1] + "_" + item_split[2] + "_" + item_split[3] + "_0_" + appendix + ".tif"
#                 name_1 = item_split[1] + "_" + item_split[2] + "_" + item_split[3] + "_1_" + appendix + ".tif"
                name_0 = item_split[0] + "_" + item_split[1] + "_" + item_split[2] + "_0_" + appendix + ".tif"
                name_1 = item_split[0] + "_" + item_split[1] + "_" + item_split[2] + "_1_" + appendix + ".tif"

#                 name_0 = item_split[0] + "_" + item_split[1] + "_0_" + appendix + ".tif"
#                 name_1 = item_split[0] + "_" + item_split[1] + "_1_" + appendix + ".tif"
                result_name = item_split[0] + "_" + item_split[1] + "_" + item_split[2] + "_" + appendix + "_Avg.tif"

                Input_1 = os.path.join(file_path, name_0)
                Input_2 = os.path.join(file_path, name_1)
                Result_raster = os.path.join(file_path, result_name)

                # To allow overwriting outputs change overwriteOutput option to True.
                arcpy.env.overwriteOutput = True

                # Check out any necessary licenses.
                arcpy.CheckOutExtension("spatial")

                # Process: Raster Calculator (Raster Calculator) (ia)
                projection = "PROJCS['S-JTSK_Krovak_East_North',GEOGCS['GCS_S_JTSK',DATUM['D_S_JTSK',SPHEROID['Bessel_1841',6377397.155,299.1528128]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Krovak'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Pseudo_Standard_Parallel_1',78.5],PARAMETER['Scale_Factor',0.9999],PARAMETER['Azimuth',30.28813975277778],PARAMETER['Longitude_Of_Center',24.83333333333333],PARAMETER['Latitude_Of_Center',49.5],PARAMETER['X_Scale',-1.0],PARAMETER['Y_Scale',1.0],PARAMETER['XY_Plane_Rotation',90.0],UNIT['Meter',1.0]]"

                # Set local variables
                in_raster1 = "raster1.tif"
                in_raster2 = "raster2.tif"

                # Excuate RasterCalculator(Minus) function
                raster_calc = RasterCalculator([Input_1, Input_2], ["x", "y"], "(x+y)/2", "FirstOf", "FirstOf")

                arcpy.management.DefineProjection(raster_calc, projection)

                # Save the output
                raster_calc.save(Result_raster)
                print("Raster ", result_name, " done")
                
        except:
            print(item, "error")
            error_list.append(item)
            
    print("Appendix", appendix, "finished")
print("Finished")
print("Errors", error_list)




Raster  ds_20220713_01_rill_Avg.tif  done
Raster  ds_20220713_02_rill_Avg.tif  done
Appendix rill finished
Raster  ds_20220713_01_no_Avg.tif  done
Raster  ds_20220713_02_no_Avg.tif  done
Appendix no finished
Raster  ds_20220713_01_sheet_Avg.tif  done
Raster  ds_20220713_02_sheet_Avg.tif  done
Appendix sheet finished
Finished
Errors []


In [30]:
# import libraries
import os
import sys
import arcpy
import numpy as np
import pandas as pd
from arcpy.sa import *
import csv

## get metadata type from metadata string
def get_metadata(type_of_data, mtd_string, separator):
    mtd_list = mtd_string.split(separator)
    for item in mtd_list:
        if type_of_data in item:
            elements = item.split(':')
            return elements[1]

# load log and create raster
def GLCMcreateRillRaster(name, folderPath, output_folder):
    # log name    
    
#     name = "sorted_ds_20201102_01_1_log.csv"
    # mtd_name = "MTD_GLCM_ds_20201102_01_0.txt"
    # folder adress

#     folderPath = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\modely\model_008\mosaics"
#     output_folder = os.path.join(folderPath.replace('\mosaics',''), 'result_rasters')
    logPath = os.path.join(folderPath, name)

    # load csv
    log = []
    with open(logPath , newline = '',encoding= 'utf-8') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=';', quotechar='"')
        for row in csv_reader:
            log.append(row)

    newList = []
    for line in log:
        if line == []:
            pass
        else:
            newList.append(line[0].split(','))

    log = newList

    ## load metadata
    mtd_name = 'MTD_' + name
    mtd_name = mtd_name.replace('sorted','GLCM')
    mtd_name = mtd_name.replace('_log','')
    mtd_name = mtd_name.replace('.csv','.txt')
    mtd_file = open(os.path.join(folderPath, mtd_name), "r")
    mtd = mtd_file.read()

    loc = get_metadata('Location', mtd, ',')
    pixel_number = int(get_metadata('Pixel Number', mtd, ','))

    cellSize = float(get_metadata('Cell Size', mtd, ','))
    # cellSize = float(log[0][1])
    Cell_Size = str(cellSize)+" "+str(cellSize)

    lowerLeft = "-1,30104260698261E-18 -4,99999999999998E-02 NaN NaN"
    lowerLeft = lowerLeft.split(' ')
    lowerLeft = arcpy.Point(lowerLeft[0], lowerLeft[1])

    dimensions = get_metadata('Dimension', mtd, ',')
    dimensions = "(42, 2025, 536)"
    dimensions = dimensions.replace('(', '')
    dimensions = dimensions.replace(')', '')
    dimensions = dimensions.split(',')

    temp = []
    for item in dimensions:
        temp.append(int(item))
    dimensions = temp

    shift = int(get_metadata('Shift', mtd, ','))

    # create empty raster
    # classArray = numpy.zeros((dimensions[1], dimensions[2]))
    classArray = numpy.full((dimensions[1], dimensions[2]), 5)

    # recognize shift
    s, t = 0, 0
    if shift == 1:
        s = int(pixel_number/2)
        t = int(pixel_number/2)

    # iterate over the log and fil the raster
    i = 0
    classification = 0

    for line in log:
        if i == 0:
            pass
        else:
    #         print(line)
            explode_name = line[0]
            explode_name = explode_name.split('_')
            coor = explode_name[4]
            coor = coor.replace('Y', 'X')
            coor = coor.split('X')
            X = int(coor[1])
            Y = int(coor[2])
            classification = int(line[1])
    #             print(result)

            k = Y * pixel_number + s
            l = X * pixel_number + t
            classArray[k:k+pixel_number, l:l+pixel_number] = classification

    #         print(coor)
    #         break

        i += 1

    #     return(output_array)
    classRaster = arcpy.NumPyArrayToRaster(classArray,lowerLeft,cellSize, value_to_nodata=255)

    # Process: Raster Calculator (Raster Calculator) (ia)
    # projection = "PROJCS['S-JTSK_Krovak_East_North',GEOGCS['GCS_S_JTSK',DATUM['D_S_JTSK',SPHEROID['Bessel_1841',6377397.155,299.1528128]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Krovak'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Pseudo_Standard_Parallel_1',78.5],PARAMETER['Scale_Factor',0.9999],PARAMETER['Azimuth',30.28813975277778],PARAMETER['Longitude_Of_Center',24.83333333333333],PARAMETER['Latitude_Of_Center',49.5],PARAMETER['X_Scale',-1.0],PARAMETER['Y_Scale',1.0],PARAMETER['XY_Plane_Rotation',90.0],UNIT['Meter',1.0]]"
    # arcpy.management.DefineProjection(classRaster, projection)

    classRasName = name.replace('_log.csv', '_class.tif')
    classRasName = classRasName.replace('sorted_', '')

    print(classRasName)

    classRaster.save(os.path.join(output_folder, classRasName))

    print('Rill Raster for ', name, ' done')     

In [32]:
log_list = ["sorted_ds_20201102_02_0_log.csv", "sorted_ds_20201102_02_1_log.csv", 
            "sorted_ds_20201102_03_0_log.csv", "sorted_ds_20201102_03_1_log.csv", "sorted_ds_20201102_04_0_log.csv", "sorted_ds_20201102_04_1_log.csv",
           "sorted_ds_20201102_05_0_log.csv", "sorted_ds_20201102_05_1_log.csv", "sorted_ds_20201102_06_0_log.csv", "sorted_ds_20201102_06_1_log.csv",]

folderPath = r"S:\\Private\\_PROJEKTY\\2017_Strix_svahy\\Tejkl_ML\\machine_learning\\modely\\model_008\\mosaics"
output_folder = r"S:\\Private\\_PROJEKTY\\2017_Strix_svahy\\Tejkl_ML\\machine_learning\\modely\\model_008\\result_rasters"

for item in log_list:
    GLCMcreateRillRaster(item, folderPath, output_folder)
#     classRasName = item.replace('_log.csv', '_class.tif')
#     classRasName = classRasName.replace('sorted_', '')

#     print(classRasName)

ds_20201102_02_0_class.tif
Rill Raster for  sorted_ds_20201102_02_0_log.csv  done
ds_20201102_02_1_class.tif
Rill Raster for  sorted_ds_20201102_02_1_log.csv  done
ds_20201102_03_0_class.tif
Rill Raster for  sorted_ds_20201102_03_0_log.csv  done
ds_20201102_03_1_class.tif
Rill Raster for  sorted_ds_20201102_03_1_log.csv  done
ds_20201102_04_0_class.tif
Rill Raster for  sorted_ds_20201102_04_0_log.csv  done
ds_20201102_04_1_class.tif
Rill Raster for  sorted_ds_20201102_04_1_log.csv  done
ds_20201102_05_0_class.tif
Rill Raster for  sorted_ds_20201102_05_0_log.csv  done
ds_20201102_05_1_class.tif
Rill Raster for  sorted_ds_20201102_05_1_log.csv  done
ds_20201102_06_0_class.tif
Rill Raster for  sorted_ds_20201102_06_0_log.csv  done
ds_20201102_06_1_class.tif
Rill Raster for  sorted_ds_20201102_06_1_log.csv  done


In [5]:
# import libraries
import os
import sys
import arcpy
import numpy as np
import pandas as pd
from arcpy.sa import *
import csv
from PIL import Image

# import special libraries
from skimage import feature
from skimage.filters import meijering, sato, frangi, hessian

def ridgeAnalysis(i, r):
    gdbPath = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\STRIX_ML.gdb"
    # storage_folder = r"S:\Private\_PROJEKTY\2020_TACR_DPZ\mapa_udalosti\Tejkl_reseni"
    window = 50
    modification = 'none'
    composite = "DS\\ds_20201126_puda_04.tif"
    composite_ras = arcpy.Raster(composite)
    shift = "0"

    # analyse it
    ## cellsize specification
    cellSize = composite_ras.meanCellWidth
    Cell_Size = str(cellSize)+" "+str(cellSize)

    ## additional values specification
    lowerLeft = arcpy.Point(composite_ras.extent.XMin,composite_ras.extent.YMin)

    ## run create mosaic not shifted
    ## export to numpy array
    composite_array = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)

    # name mosaic, depending on calibration layer
    loc = "ds_20201126_puda_04"
#     i = 2
    i = str(i)
#     r = 10

    ## array dimensions
    dimensions = composite_array.shape

    # create cell mosaic
    cell_meters = window   # width of cell in meters

    ## copy cell into new raster
    pixel_number = int(cell_meters/cellSize)   # pixel number equals to width of cell
    pixel_area = pixel_number * pixel_number   # area of cell in pixels

    arr_max = []
    ## array maximums
    for band in composite_array:
        arr_max.append(numpy.amax(band))

    del(arr_max[0])

    ## print informations
    print(cellSize, lowerLeft, dimensions, pixel_number, pixel_area, )

    ## create greyscale array and append it to composite array
    # greyscale_array = (0.3 * Red) + (0.59 * Green) + (0.11 * Blue)
    greyscale_array = ((0.3 * composite_array[3]) + (0.59 * composite_array[2]) + (0.11 * composite_array[1]))
    composite_array = np.append(composite_array, [greyscale_array], axis=0)

    meijeArray = meijering(greyscale_array, sigmas=range(1, r, 2), alpha=None, black_ridges=True, mode='reflect', cval=0)
    satoArray = sato(greyscale_array, sigmas=range(1, r, 2), black_ridges=True, mode='reflect', cval=0)
    frangiArray = frangi(greyscale_array, sigmas=range(1, r, 2), scale_range=None, scale_step=None, alpha=0.5, beta=0.5, gamma=15, black_ridges=True, mode='reflect', cval=0)
    hessianArray = hessian(greyscale_array, sigmas=range(1, r, 2), scale_range=None, scale_step=None, alpha=0.5, beta=0.5, gamma=15, black_ridges=True, mode='reflect', cval=0)

    meijeRaster = arcpy.NumPyArrayToRaster(meijeArray, lowerLeft, cellSize)
    meijeRaster.save(os.path.join(gdbPath, "meijeRaster" + i))

    satoRaster = arcpy.NumPyArrayToRaster(satoArray, lowerLeft, cellSize)
    satoRaster.save(os.path.join(gdbPath, "satoRaster" + i))

    frangiRaster = arcpy.NumPyArrayToRaster(frangiArray, lowerLeft, cellSize)
    frangiRaster.save(os.path.join(gdbPath, "frangiRaster" + i))

    hessianRaster = arcpy.NumPyArrayToRaster(hessianArray, lowerLeft, cellSize)
    hessianRaster.save(os.path.join(gdbPath, "hessianRaster" + i))

    print("i =", i , 'range', r)

In [2]:
# i = 1 range 50
# i = 2 range 10
# i = 3 range 20
# i = 4 range 30
# i = 5 range 40
# i = 6 range 100

In [6]:
rList = [10, 20, 30, 40, 100]

i = 2
for item in rList:
    ridgeAnalysis(i, item)
    i += 1

0.001 0 -4,90000000000004E-02 NaN NaN (4, 4049, 1071) 50000 2500000000
i =  2 range 10
0.001 0 -4,90000000000004E-02 NaN NaN (4, 4049, 1071) 50000 2500000000
i =  3 range 20
0.001 0 -4,90000000000004E-02 NaN NaN (4, 4049, 1071) 50000 2500000000
i =  4 range 30
0.001 0 -4,90000000000004E-02 NaN NaN (4, 4049, 1071) 50000 2500000000
i =  5 range 40
0.001 0 -4,90000000000004E-02 NaN NaN (4, 4049, 1071) 50000 2500000000
i =  6 range 100


In [57]:
# import libraries
import os
import sys
import arcpy
import numpy as np
import pandas as pd
from arcpy.sa import *
import csv
from PIL import Image

# import special libraries
from skimage import measure

# def contourAnalysis(i, lev, item, element):
i = 0
lev = 200
item = "low"
element = "low"

gdbPath = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\STRIX_ML.gdb"
composite = "DS\\ds_20201126_puda_04.tif"
composite_ras = arcpy.Raster(composite)

# analyse it
## cellsize specification
cellSize = composite_ras.meanCellWidth
Cell_Size = str(cellSize)+" "+str(cellSize)

## additional values specification
lowerLeft = arcpy.Point(composite_ras.extent.XMin,composite_ras.extent.YMin)

## run create mosaic not shifted
## export to numpy array
composite_array = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)

# name mosaic, depending on calibration layer
loc = "ds_20201126_puda_04"

i = str(i)

## array dimensions
dimensions = composite_array.shape

# create cell mosaic
cell_meters = window   # width of cell in meters

## copy cell into new raster
pixel_number = int(cell_meters/cellSize)   # pixel number equals to width of cell
pixel_area = pixel_number * pixel_number   # area of cell in pixels

arr_max = []
## array maximums
for band in composite_array:
    arr_max.append(numpy.amax(band))

del(arr_max[0])

## print informations
#     print(cellSize, lowerLeft, dimensions, pixel_number, pixel_area, )

## create greyscale array and append it to composite array
# greyscale_array = (0.3 * Red) + (0.59 * Green) + (0.11 * Blue)
greyscale_array = ((0.3 * composite_array[3]) + (0.59 * composite_array[2]) + (0.11 * composite_array[1]))
composite_array = np.append(composite_array, [greyscale_array], axis=0)

contoursArray = measure.find_contours(greyscale_array, level = lev, fully_connected=item, positive_orientation=element)

for contour in contoursArray:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

ax.axis('image')
ax.set_xticks([])
ax.set_yticks([])
plt.show()

#     contoursRaster = arcpy.NumPyArrayToRaster(contoursArray, lowerLeft, cellSize)
#     contoursRaster.save(os.path.join(gdbPath, "contoursRas" + i))
#     print(contoursArray)
print("i =", i , "fully_connected", item, "positive_orientation", element)

i = 0 fully_connected low positive_orientation low


In [52]:
aList = ["low", "high"]
bList = ["low", "high"]
cList = [350, 400]

i = 1
for item in aList:
    for element in bList:
            for lev in cList:
                contourAnalysis(i, lev, item, element)
                i += 1


i = 1 fully_connected low positive_orientation low
i = 2 fully_connected low positive_orientation low
i = 3 fully_connected low positive_orientation high
i = 4 fully_connected low positive_orientation high
i = 5 fully_connected high positive_orientation low
i = 6 fully_connected high positive_orientation low
i = 7 fully_connected high positive_orientation high
i = 8 fully_connected high positive_orientation high


In [3]:
# import libraries
import os
import sys
import arcpy
import numpy as np
import pandas as pd
from arcpy.sa import *
import csv
from PIL import Image

# import special libraries
from skimage import data
from skimage.feature import blob_dog, blob_log, blob_doh

def blobAnalysis(minS, maxS, numS):

    gdbPath = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\STRIX_ML.gdb"
    composite = "DS\\ds_20201126_puda_04.tif"
    composite_ras = arcpy.Raster(composite)

    # analyse it
    ## cellsize specification
    cellSize = composite_ras.meanCellWidth
    Cell_Size = str(cellSize)+" "+str(cellSize)

    ## additional values specification
    lowerLeft = arcpy.Point(composite_ras.extent.XMin,composite_ras.extent.YMin)

    ## run create mosaic not shifted
    ## export to numpy array
    composite_array = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)

    ## array dimensions
    dimensions = composite_array.shape

    arr_max = []
    ## array maximums
    for band in composite_array:
        arr_max.append(numpy.amax(band))

    del(arr_max[0])

    ## print informations
    #     print(cellSize, lowerLeft, dimensions, pixel_number, pixel_area, )

    ## create greyscale array and append it to composite array
    # greyscale_array = (0.3 * Red) + (0.59 * Green) + (0.11 * Blue)
    greyscale_array = ((0.3 * composite_array[3]) + (0.59 * composite_array[2]) + (0.11 * composite_array[1]))
    composite_array = np.append(composite_array, [greyscale_array], axis=0)
    composite_array = composite_array[0:100, 0:100]

    #  superparameters
#     minS = 1
#     maxS = 50
#     Sratio = 1.6
    numS = 10
    tresh = 0.01
    overlp = 0.5
    
    print("jdeme na to")

#     blobList = blob_dog(greyscale_array, min_sigma= minS, max_sigma= maxS, sigma_ratio= Sratio, threshold= tresh, overlap= overlp)
    blobList = blob_doh(greyscale_array, min_sigma= minS, max_sigma= maxS, num_sigma= numS, threshold= tresh, overlap= overlp)
#     blob_log(image, min_sigma=1, max_sigma=50, num_sigma=10, threshold=0.2, overlap=0.5, log_scale=False, *, threshold_rel=None, exclude_border=False)

    print("bloby done")
    
    pointName = 'Blob'
    name = 'blob_name' 
    
    layerName = 'doh' + str(minS) + '_' + str(maxS) + '_' + str(int(Sratio*10))
    altGdb = 'S:/Private/_PROJEKTY/2017_Strix_svahy/Tejkl_ML/machine_learning/pokus_point.gdb'
    layerPath = os.path.join(altGdb, layerName)
    
    arcpy.CreateFeatureclass_management(altGdb, layerName, "POINT")
    arcpy.AddField_management(layerPath, name, "TEXT", "", "", "", "", "", "", "")

    # Open an InsertCursor
    cursor = arcpy.da.InsertCursor(layerPath, [name, 'SHAPE@XY'])

    row_values = []
    temp = []
    i = 0
    # create rows
    for item in blobList:
        temp.append(pointName + str(i))
        temp.append(arcpy.Point(item[1]/1000, item[0]/1000))
        i += 1
        row_values.append(temp)
        temp = []

    print(row_values[-1])
    # Insert new rows that include the county name and a x,y coordinate
    #  pair that represents the county center
    for row in row_values:
        cursor.insertRow(row)

    # Delete cursor object
    del cursor

    print(layerName, 'done')

In [4]:
suParamList = [[10,50,10],[20,50,20],[50,100, 10],[2,100, 50],[1,30,10],]

for item in suParamList:
    blobAnalysis(item[0] , item[1] , item[2] )

jdeme na to


MemoryError: 

In [1]:
# import libraries
import os
import sys
import arcpy
import numpy as np
from arcpy.sa import *
import GLCM



def createGLCMmosaics(loc, pixel_number, cellSize, composite_array, shift, just_others):
    
    # verze "just_others" [ryhy, blue, green, red, alpha]
    # verze "just_others" [blue, green, red, alpha]
    A, B = 0, 0
    ## raw composite array dimensions
    dimensions = composite_array.shape
    ## delete pseudo alpha channel
    composite_array = np.delete(composite_array, dimensions[0]-1, 0)

    ## copy cell into new raster
#     pixel_number = int(cell_meters/cellSize)   # pixel number equals to width of cell
    cell_area = pixel_number * pixel_number   # number of pixels in cell
        
    ## create greyscale array and append it to composite array
    if just_others == "just_others":
        red, green, blue = 0, 1, 2
    else:
        red, green, blue = 1, 2, 3

    # greyscale_array = (0.3 * Red) + (0.59 * Green) + (0.11 * Blue)
    greyscale_array = ((0.3 * composite_array[red]) + (0.59 * composite_array[green]) + (0.11 * composite_array[blue]))
#     composite_array = np.append(composite_array, [greyscale_array], axis=0)
        
    # Compute the Canny filter for two values of sigma
    
    ## array dimensions
    dimensions = composite_array.shape
    
    ## print informations
    print(cellSize, lowerLeft, dimensions, pixel_number, cell_area, )
    
    ## array maximums
    i = 0
    arr_max = []
    if just_others == "just_others":
        for item in composite_array:
            arr_max.append(numpy.amax(composite_array[i]))
            i += 1
    else:
        for item in composite_array:
            if i == 0:
                i += 1
            else:
                arr_max.append(numpy.amax(composite_array[i]))
                i += 1
                
    print(arr_max)
    
    ## create log file
    log = [[loc, cellSize, lowerLeft, pixel_number, dimensions, shift]]
    print(log[0])
    ## calculate for loop
    y = dimensions[1]
    x = dimensions[2]

    x_steps = int(x/pixel_number)
    y_steps = int(y/pixel_number)

    if x_steps*pixel_number > x:
        x_steps = x_steps - 1

    if y_steps*pixel_number > y:
        y_steps = y_steps - 1
        
    if shift == 1:
        x_steps = x_steps - 1
        y_steps = y_steps - 1

    for_range = x_steps * y_steps
    print('For range ', for_range)
    A = 0
#     for_range = 80

    print('Empty mosaic array shape ',greyscale_array.shape)
#     print(mosaic_array)

    # create empty rasters
    res_con_v = numpy.zeros((dimensions[1], dimensions[2]))
    res_dis_v = numpy.zeros((dimensions[1], dimensions[2]))
    res_hom_v = numpy.zeros((dimensions[1], dimensions[2]))
    res_sim_v = numpy.zeros((dimensions[1], dimensions[2]))
    res_mean_v = numpy.zeros((dimensions[1], dimensions[2]))
    res_vari_v = numpy.zeros((dimensions[1], dimensions[2]))
    res_corre_v = numpy.zeros((dimensions[1], dimensions[2]))
    
    res_con_h = numpy.zeros((dimensions[1], dimensions[2]))
    res_dis_h = numpy.zeros((dimensions[1], dimensions[2]))
    res_hom_h = numpy.zeros((dimensions[1], dimensions[2]))
    res_sim_h = numpy.zeros((dimensions[1], dimensions[2]))
    res_mean_h = numpy.zeros((dimensions[1], dimensions[2]))
    res_vari_h = numpy.zeros((dimensions[1], dimensions[2]))
    res_corre_h = numpy.zeros((dimensions[1], dimensions[2]))
    
    counter = 0
    k = 0
    l = 0
    index = 1
    s = 0
    t = 0
    i = 0
    x = 0
    y = 0
    
    if shift == 1:
        k = int(pixel_number/2)
        l = int(pixel_number/2)
#         print(shift, k, l)

    for step in range(for_range):
        if just_others == "just_others":
            start_band = 0
            n = 0
            arr_count = 0
        else:
            start_band = 1
            n = 1
            arr_count = 0
            
#   calculate GLCM for clip of greyscale array
        kernel = greyscale_array[k:k+pixel_number, l:l+pixel_number]
        uniq = GLCM.gen_unique(kernel)

        GLCM_v = GLCM.vertical_GLCM(kernel, uniq)
        GLCM_h = GLCM.horizontal_GLCM(kernel, uniq)

        # vertical computation
        con = GLCM.contrast(GLCM_v)
        dis = GLCM.dissimilarity(GLCM_v)
        hom = GLCM.homogenity(GLCM_v)
        sim = GLCM.similarity(GLCM_v)
        mean = GLCM.GLCM_mean(GLCM_v)
        vari = GLCM.variance(GLCM_v, mean)
        corre = GLCM.correlation(GLCM_v, mean, vari)

        res_con_v[k:k+pixel_number, l:l+pixel_number] = con
        res_dis_v[k:k+pixel_number, l:l+pixel_number] = dis
        res_hom_v[k:k+pixel_number, l:l+pixel_number] = hom
        res_sim_v[k:k+pixel_number, l:l+pixel_number] = sim
        res_mean_v[k:k+pixel_number, l:l+pixel_number] = mean
        res_vari_v[k:k+pixel_number, l:l+pixel_number] = vari
        res_corre_v[k:k+pixel_number, l:l+pixel_number] = corre
        
        # horizontal computation    
        con = GLCM.contrast(GLCM_h)
        dis = GLCM.dissimilarity(GLCM_h)
        hom = GLCM.homogenity(GLCM_h)
        sim = GLCM.similarity(GLCM_h)
        mean = GLCM.GLCM_mean(GLCM_h)
        vari = GLCM.variance(GLCM_h, mean)
        corre = GLCM.correlation(GLCM_h, mean, vari)

        res_con_h[k:k+pixel_number, l:l+pixel_number] = con
        res_dis_h[k:k+pixel_number, l:l+pixel_number] = dis
        res_hom_h[k:k+pixel_number, l:l+pixel_number] = hom
        res_sim_h[k:k+pixel_number, l:l+pixel_number] = sim
        res_mean_h[k:k+pixel_number, l:l+pixel_number] = mean
        res_vari_h[k:k+pixel_number, l:l+pixel_number] = vari
        res_corre_h[k:k+pixel_number, l:l+pixel_number] = corre
        
        if just_others == "just_others":
            state_noPix = 0
            edge_array = composite_array[0][k:k+pixel_number, l:l+pixel_number]
        
        x += 1
        s = 0
        t = 0
        l = l + pixel_number

        # jump to new line
        if x == x_steps:
            x = 0
            l = 0
            k = k + pixel_number
            y += 1
            
            # recognize shift
            if shift == 1:
                l = int(pixel_number/2)
        
        counter += 1
#         percento = [10, 20, 30, 40, 50, 60, 70, 80, 90]
        percento = [20, 40, 60, 80, 99]
        if int(counter) in percento:
            print(int(counter),'% done')
                
    con_v_raster = arcpy.NumPyArrayToRaster(res_con_v,lowerLeft,cellSize,value_to_nodata=255)
    dis_v_raster = arcpy.NumPyArrayToRaster(res_dis_v,lowerLeft,cellSize,value_to_nodata=255)
    hom_v_raster = arcpy.NumPyArrayToRaster(res_hom_v,lowerLeft,cellSize,value_to_nodata=255)
    sim_v_raster = arcpy.NumPyArrayToRaster(res_sim_v,lowerLeft,cellSize,value_to_nodata=255)
    mean_v_raster = arcpy.NumPyArrayToRaster(res_mean_v,lowerLeft,cellSize,value_to_nodata=255)
    vari_v_raster = arcpy.NumPyArrayToRaster(res_vari_v,lowerLeft,cellSize,value_to_nodata=255)
    corre_v_raster = arcpy.NumPyArrayToRaster(res_corre_v,lowerLeft,cellSize,value_to_nodata=255)

    con_h_raster = arcpy.NumPyArrayToRaster(res_con_h,lowerLeft,cellSize,value_to_nodata=255)
    dis_h_raster = arcpy.NumPyArrayToRaster(res_dis_h,lowerLeft,cellSize,value_to_nodata=255)
    hom_h_raster = arcpy.NumPyArrayToRaster(res_hom_h,lowerLeft,cellSize,value_to_nodata=255)
    sim_h_raster = arcpy.NumPyArrayToRaster(res_sim_h,lowerLeft,cellSize,value_to_nodata=255)
    mean_h_raster = arcpy.NumPyArrayToRaster(res_mean_h,lowerLeft,cellSize,value_to_nodata=255)
    vari_h_raster = arcpy.NumPyArrayToRaster(res_vari_h,lowerLeft,cellSize,value_to_nodata=255)
    corre_h_raster = arcpy.NumPyArrayToRaster(res_corre_h,lowerLeft,cellSize,value_to_nodata=255)
    
    print("Numpy to raster done")

    # Process: Composite Bands (Composite Bands) (management)
    GLCM_vertical_raster = os.path.join(gdbPath, loc + "_GLCM_v_" + str(shift))
    raster_list = [con_v_raster, dis_v_raster, hom_v_raster, sim_v_raster, mean_v_raster, vari_v_raster, corre_v_raster]
#     raster_list = [con_v_raster, dis_v_raster]
    with arcpy.EnvManager(scratchWorkspace= gdbPath, workspace= gdbPath):
        arcpy.management.CompositeBands(in_rasters = raster_list, out_raster = GLCM_vertical_raster)
        
    print("Composite v done")

    GLCM_horizontal_raster = os.path.join(gdbPath, loc + "_GLCM_h_" + str(shift))
    raster_list = [con_h_raster, dis_h_raster, hom_h_raster, sim_h_raster, mean_h_raster, vari_h_raster, corre_h_raster]
#     raster_list = [con_h_raster, dis_h_raster]
    with arcpy.EnvManager(scratchWorkspace= gdbPath, workspace= gdbPath):
        arcpy.management.CompositeBands(in_rasters = raster_list, out_raster = GLCM_horizontal_raster)
    
    print("Mosaics ", loc, ' shift ', shift, " done")
    print()

In [4]:
# import libraries
import os
import sys
import arcpy
import numpy as np
import pandas as pd
from arcpy.sa import *
from datetime import datetime

pool = [ "ds_20220309_puda_synt_res2maj"]

# window_list = [ 10]
window_list = [ 5, 8, 10]

# take composite
# loc = "Litultovice_20190811"
gdbPath = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\STRIX_GLCM.gdb"
verze = "just_others"

for item in pool:
    for window in window_list:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("Start Time =", current_time)

#         loc = pool[0]
    #     composite = "DS\\" + loc + ".tif"
        composite = "synteticke_ryhy\\" + item
        composite_ras = arcpy.Raster(composite)
        name = item.split("_")
        loc = name[2] + "_" + name[1] + "_" + name[3] + "_" + str(window)

        # analyse it
        ## cellsize specification
        cellSize = composite_ras.meanCellWidth
        Cell_Size = str(cellSize)+" "+str(cellSize)

        ## additional values specification
        lowerLeft = arcpy.Point(composite_ras.extent.XMin,composite_ras.extent.YMin)

        ## run create mosaic not shifted
        ## export to numpy array
        composite_array_0 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
        createGLCMmosaics(loc, window, cellSize, composite_array_0, 0, verze)
        
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("Middle Time =", current_time)

        ## run create mosaic shifted
        ## export to numpy array
        composite_array_1 = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)
        createGLCMmosaics(loc, window, cellSize, composite_array_1, 1, verze)

        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("End Time =", current_time)

Start Time = 12:31:52
0.002 -4,33680868994202E-19 -4,99999999999998E-02 NaN NaN (3, 2025, 536) 5 25
[253.0, 253.0, 254.0]
['puda_20220309_synt_5', 0.002, <Point (-4.336808689942018e-19, -0.04999999999999982, #, #)>, 5, (3, 2025, 536), 0]
For range  43335
Empty mosaic array shape  (2025, 536)
20 % done
40 % done
60 % done
80 % done
99 % done
Numpy to raster done
Composite v done
Mosaics  puda_20220309_synt_5  shift  0  done

Middle Time = 14:43:32
0.002 -4,33680868994202E-19 -4,99999999999998E-02 NaN NaN (3, 2025, 536) 5 25
[253.0, 253.0, 254.0]
['puda_20220309_synt_5', 0.002, <Point (-4.336808689942018e-19, -0.04999999999999982, #, #)>, 5, (3, 2025, 536), 1]
For range  42824
Empty mosaic array shape  (2025, 536)
20 % done
40 % done
60 % done
80 % done
99 % done
Numpy to raster done
Composite v done
Mosaics  puda_20220309_synt_5  shift  1  done

End Time = 16:54:40
Start Time = 16:54:40
0.002 -4,33680868994202E-19 -4,99999999999998E-02 NaN NaN (3, 2025, 536) 8 64
[253.0, 253.0, 254.0]
[

In [9]:
# import libraries
import os
import sys
import arcpy
import numpy as np
from arcpy.sa import *
import GLCM
from datetime import datetime

def timeGLCMmosaics(pixel_number, composite_array):
    
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Start Time =", current_time)

    # verze "just_others" [ryhy, blue, green, red, alpha]
    # verze "just_others" [blue, green, red, alpha]
    A, B = 0, 0
    ## raw composite array dimensions
    dimensions = composite_array.shape
    ## delete pseudo alpha channel
    composite_array = np.delete(composite_array, dimensions[0]-1, 0)

    ## copy cell into new raster
    #     pixel_number = int(cell_meters/cellSize)   # pixel number equals to width of cell
    cell_area = pixel_number * pixel_number   # number of pixels in cell

    ## create greyscale array and append it to composite array

    red, green, blue = 0, 1, 2


    # greyscale_array = (0.3 * Red) + (0.59 * Green) + (0.11 * Blue)
    greyscale_array = ((0.3 * composite_array[red]) + (0.59 * composite_array[green]) + (0.11 * composite_array[blue]))
    #     composite_array = np.append(composite_array, [greyscale_array], axis=0)

    # Compute the Canny filter for two values of sigma


    k = 250
    l = 250

    ## array dimensions
    dimensions = composite_array.shape

    ## print informations
    print(  dimensions, pixel_number, cell_area, )

    ## create log file
    log = [[  pixel_number, dimensions]]
    print(log[0])
    ## calculate for loop
    y = dimensions[1]
    x = dimensions[2]

    x_steps = int(x/pixel_number)
    y_steps = int(y/pixel_number)

    if x_steps*pixel_number > x:
        x_steps = x_steps - 1

    if y_steps*pixel_number > y:
        y_steps = y_steps - 1

    for_range = x_steps * y_steps
    print('For range ', for_range)
    A = 0
    #     for_range = 80

    print('Empty mosaic array shape ',greyscale_array.shape)
    #     print(mosaic_array)

    # create empty rasters
    res_con_v = numpy.zeros((dimensions[1], dimensions[2]))
    res_dis_v = numpy.zeros((dimensions[1], dimensions[2]))
    res_hom_v = numpy.zeros((dimensions[1], dimensions[2]))
    res_sim_v = numpy.zeros((dimensions[1], dimensions[2]))
    res_mean_v = numpy.zeros((dimensions[1], dimensions[2]))
    res_vari_v = numpy.zeros((dimensions[1], dimensions[2]))
    res_corre_v = numpy.zeros((dimensions[1], dimensions[2]))

    res_con_h = numpy.zeros((dimensions[1], dimensions[2]))
    res_dis_h = numpy.zeros((dimensions[1], dimensions[2]))
    res_hom_h = numpy.zeros((dimensions[1], dimensions[2]))
    res_sim_h = numpy.zeros((dimensions[1], dimensions[2]))
    res_mean_h = numpy.zeros((dimensions[1], dimensions[2]))
    res_vari_h = numpy.zeros((dimensions[1], dimensions[2]))
    res_corre_h = numpy.zeros((dimensions[1], dimensions[2]))

    print("rasters created")

    #   calculate GLCM for clip of greyscale array
    kernel = greyscale_array[k:k+pixel_number, l:l+pixel_number]
    uniq = GLCM.gen_unique(kernel)

    print("GLCM unique created")

    GLCM_v = GLCM.vertical_GLCM(kernel, uniq)
    GLCM_h = GLCM.horizontal_GLCM(kernel, uniq)

    print("GLCM v and h created")

    # vertical computation
    con = GLCM.contrast(GLCM_v)
    dis = GLCM.dissimilarity(GLCM_v)
    hom = GLCM.homogenity(GLCM_v)
    sim = GLCM.similarity(GLCM_v)
    mean = GLCM.GLCM_mean(GLCM_v)
    vari = GLCM.variance(GLCM_v, mean)
    corre = GLCM.correlation(GLCM_v, mean, vari)

    # horizontal computation    
    con = GLCM.contrast(GLCM_h)
    dis = GLCM.dissimilarity(GLCM_h)
    hom = GLCM.homogenity(GLCM_h)
    sim = GLCM.similarity(GLCM_h)
    mean = GLCM.GLCM_mean(GLCM_h)
    vari = GLCM.variance(GLCM_h, mean)
    corre = GLCM.correlation(GLCM_h, mean, vari)

    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("End Time =", current_time)
    print("GLCM with side", pixel_number , " done")
    print()

In [14]:
# import libraries
import os
import sys
import arcpy
import numpy as np
from arcpy.sa import *
import GLCM
from datetime import datetime

gdbPath = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\STRIX_ML.gdb"

verze = "just_others"

composite = "DS\\" + "ds_20201126_puda_04_res10maj"
composite_ras = arcpy.Raster(composite)

composite_array = arcpy.RasterToNumPyArray(composite_ras,nodata_to_value=0)

side_list = [5]
for item in side_list:
    timeGLCMmosaics(item, composite_array)
    

Start Time = 12:26:59
(3, 406, 107) 5 25
[5, (3, 406, 107)]
For range  1701
Empty mosaic array shape  (406, 107)
rasters created
GLCM unique created
GLCM v and h created
0.0
0.0
0.0
0.0
0
0
0
0.0
0.0
0.0
0.0
0
0
0
End Time = 12:26:59
GLCM with side 5  done



In [2]:
item = [0.0, 5.5, 6, 9, 120, 88, 99]
rill_alternative_list = [0.0, 88, 99]
state_noRill, state_sheet, state_rill = 0, 0, 0

for pixel in item:
    if pixel == NoRill_list[2]:
        state_noRill += 1
    elif pixel == NoRill_list[1]:
        state_sheet += 1
    elif pixel not in NoRill_list:
        state_rill += 1
    else:
        pass
    
print(state_noRill, state_sheet, state_rill)

1 1 4


In [6]:
print(error_list)


['20201112_04_J', '20201105_03_J', '20201105_03_J', '20201105_03_J', '20201105_03_J', '20201105_03_J', '20200708_02_J', '20200708_02_J']


In [26]:
item = '33_20201203//Composite_20201203_06'
loc = item.split("Composite_")
loc = loc[1]

In [27]:
loc


'20201203_06'